In [1]:
import pandas as pd
import numpy as np
import math
import random

In [2]:
filename = "sap_storing_data_hu_project.csv" 
n = sum(1 for line in open(filename, encoding='utf8'))-1  
s = n//10
skip = sorted(random.sample(range(1, n+1), n-s))
df_not_cleaned = pd.read_csv(filename, skiprows=skip, low_memory=False)
#verwijder alle kolommen die niet van gebruik zijn:
df = df_not_cleaned.drop(columns=['stm_sap_mon_meld_ddt', 'stm_mon_begin_ddt', 'stm_mon_toelichting_trdl', 'stm_oh_pg_mld', 'stm_geo_mld_uit_functiepl', 'stm_scenario_mon', 'stm_status_melding_sap', 'stm_mon_nr_status_omschr', 'stm_mon_nr__statuscode', 'stm_mon_nr_status_wijzdd', 'stm_aanntpl_ddt', 'stm_geo_gst_uit_functiepl', 'stm_objectdl_code_gst', 'stm_objectdl_groep_gst', 'stm_progfh_in_ddt', 'stm_progfh_in_invoer_ddt', 'stm_progfh_gw_ddt', 'stm_progfh_gw_lwd_ddt', 'stm_progfh_hz', 'stm_veroorz_groep', 'stm_veroorz_code', 'stm_progfh_gw_duur', 'stm_progfh_gw_lwd_tijd', 'stm_progfh_gw_lwd_datum', 'stm_progfh_gw_datum', 'stm_arbeid', 'stm_functiepl_gst', 'stm_functiepl_mld', 'stm_mon_begintijd', 'stm_mon_begindatum', 'stm_aangelegd_tijd', 'stm_aangelegd_dd', 'stm_dir_betrok_tr', 'stm_evb', 'stm_mon_vhdsincident', 'stm_mon_eind_ddt', 'stm_afspr_aanvangddt', 'stm_effect', 'stm_veroorz_tekst_kort', 'stm_projnr', 'stm_wbi_nummer', 'stm_status_gebr', 'stm_standplaats', 'stm_actie', 'stm_hinderwaarde', 'stm_rec_toegev_ddt', 'stm_afsluit_tijd', 'stm_afsluit_dd', 'stm_afsluit_ddt', 'stm_akkoord_melding_jn', 'stm_fact_jn', 'stm_status_sapnaarmon', 'stm_akkoord_mon_toewijz', 'stm_controle_dd', 'stm_mon_eind_tijd', 'stm_mon_eind_datum', 'stm_reactie_duur', 'stm_afspr_aanvangtijd', 'stm_afspr_aanvangdd', 'stm_historie_toelichting', 'stm_schade_verhaalb_jn', 'stm_x_bron_arch_dt', 'stm_x_bron_bestandsnaam', 'stm_x_bron_publ_dt', 'stm_rapportage_jaar', 'stm_rapportage_maand', 'stm_sorteerveld', 'stm_afspr_func_hersteltijd', 'stm_afspr_func_hersteldd', 'stm_relatiervo', 'stm_relatiervo_volgend', 'stm_relatiervo_vorig', 'stm_schade_statusdatum', 'stm_schade_status_ga', 'stm_schadenr', 'stm_x_start_sessie_dt', 'stm_x_bk', 'stm_x_run_id', 'stm_x_actueel_ind', 'stm_x_vervallen_ind'])
print("Done")

Done


# Data cleaning:


De te bepalen welke variabelen het meest van toepassing zijn om te gebruiken kijken we naar welke
kolommen informatie geven over de tijd van de melding, monteur. Tijd wanneer de melding word opgelost.
Wat voor melding het is en info over de melding/storing, en de locatie. Zo zijn de volgende kolommen
het meest van toepassing:

Over soort melding
- stm_sap_meldtekst (melding omschrijving)
- stm_geo_mld (melding code) - nodig
- stm_oorz_code (oorzaak code) - nodig
- stm_prioriteit (prioriteit)

Over tijd:
- stm_sap_melddatum (datump melding) - nodig
- stm_sap_meldtijd (tijdstip melding) - nodig
- stm_aanngeb_dd (datum aannemer gebeld)
- stm_aanngeb_tijd (tijd aannemer gebeld)
- stm_aanntpl_dd (datum aannemer op plek)
- stm_aanntpl_tijd (tijd aannemer op plek)

- stm_fh_dd (Datum van functie herstel) - nodig
- stm_fh_tijd (Tijdstip van functieherstel) - nodig
- stm_sap_storeinddatum (Datum einde storing) - nodig
- stm_sap_storeindtijd (Tijdstip einde storing) - nodig
- stm_fh_duur (Duur van functiehersteltijd) - nodig

Over equipment:
- stm_equipm_nr_mld (equipment)
- stm_equipm_omschr_mld (equipment omschrijving)

Over locatie
- stm_km_van_mld (locatie van)
- stm_km_tot_mld (locatie tot)
- stm_vl_post (VL post) - nodig
- stm_contractgeb_mld(locatie melding) - nodig
- stm_pplg_van (PPLG van locatie)
- stm_pplg_naar (PPLG tot locatie)
- stm_dstrglp_van (Dienstregelpunt van locatie)
- stm_dstrglp_naar (Dienstregelpunt tot locatie)

Over prognose:
- stm_progfh_in_datum (Datum prognose)
- stm_progfh_in_tijd (Tijdstip FHT prognose)
- stm_progfh_in_invoer_dat (Invoer datum prognose)
- stm_progfh_in_invoer_tijd (Invoer tijdstip FHT prognose)
- stm_progfh_in_duur (Duur FHT)



In [3]:
#deze functie verwijdert alle rijen waar geen value op de gegeven kolommen zitten
def drop_rows_with_null(lst, df):
    dfisnull = df.isnull()[lst]
    matches = dfisnull[(dfisnull).any(axis=1)]
    return df.drop(matches.index)

In [4]:
# Verwijder de rijen die geen value bevatten in de benodigde kolommen
df_filtered = drop_rows_with_null(['stm_geo_mld', 'stm_sap_melddatum', 'stm_oorz_code', 'stm_sap_meldtijd',
                          'stm_fh_dd', 'stm_fh_tijd', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd',
                          'stm_fh_duur', 'stm_vl_post', 'stm_contractgeb_mld'], df)
print(len(df))
print(len(df_filtered))
df_filtered

89855
13385


,Unnamed: 0,#stm_sap_meldnr,stm_mon_nr,stm_vl_post,stm_sap_meld_ddt,stm_sap_meldtekst_lang,stm_sap_meldtekst,stm_geo_mld,stm_equipm_nr_mld,stm_equipm_soort_mld,...,stm_fh_dd,stm_fh_tijd,stm_fh_duur,stm_sap_storeinddatum,stm_sap_storeindtijd,stm_oorz_tekst_kort,stm_pplg_van,stm_pplg_naar,stm_dstrglp_van,stm_dstrglp_naar
9,125,50053438,62.0,AH,15/01/2006 23:08:00,Logboeknr RBV : 62 Tijd: 2308 VL-Post: ...,Logboeknr RBV : 62 Tijd: 2308 VL-P,514.0,10201441.0,WISSEL,...,15/01/2006,23:20:00,12.0,15/01/2006,23:20:00,NaN,NaN,NaN,Nm,NaN
26,361,50053804,91.0,ASD,04/02/2006 13:17:00,Logboeknr RBV : 91 Tijd: 1317 VL-Post: ...,Logboeknr RBV : 91 Tijd: 1317 VL-P,523.0,10643160.0,INTLOCKUNT,...,04/02/2006,13:42:00,25.0,04/02/2006,13:42:00,Ongeplande werkzaamheden aan de EBS Ass.,NaN,NaN,Ass,NaN
41,510,50054037,27.0,AH,21/02/2006 06:09:00,Logboeknr RBV : 27 Tijd: 0609 VL-Post: ...,Logboeknr RBV : 27 Tijd: 0609 VL-P,609.0,10202010.0,WISSEL,...,21/02/2006,06:17:00,8.0,21/02/2006,06:17:00,NaN,NaN,NaN,Zpgea,NaN
42,514,50054041,197.0,ZL,20/02/2006 21:56:00,Logboeknr RBV : 197 Tijd: 2156 VL-Post: ...,Logboeknr RBV : 197 Tijd: 2156 VL-P,603.0,10201765.0,WISSEL,...,20/02/2006,23:06:00,70.0,20/02/2006,23:06:00,NaN,NaN,NaN,Zl,NaN
43,515,50054043,223.0,GVC,24/01/2006 16:30:00,Logboeknr RBV : 223 Tijd: 1630 VL-Post: ...,Logboeknr RBV : 223 Tijd: 1630 VL-P,104.0,10107573.0,BAANLICHAA,...,24/01/2006,17:12:00,42.0,24/01/2006,17:12:00,NaN,NaN,NaN,DVNK,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89790,908020,96001289,3626889.0,ZL,24/04/2016 02:41:51,Bl-Asn : ahob 39.6 op de storingsmelder,Bl-Asn : ahob 39.6 op de storingsmelder,12.0,10211043.0,OVERWEG,...,24/04/2016,03:49:00,9.0,24/04/2016,03:50:00,Gestoord object: OVERWEG 39.6. Gestoord,BL,NaN,BL,ASN
89793,908058,98000478,160.0,UT,25/04/2006 17:58:00,Ut TOBS in wsl-sectie 1203A / 1303B..,Ut TOBS in wsl-sectie 1203A / 1303B..,531.0,10302665.0,DETGRSSSL,...,25/04/2006,19:00:00,60.0,25/04/2006,19:00:00,Staalsplinter over ES-las sectie 1203B,UT,NaN,UT,NaN
89796,908071,99002816,71.0,KFH,08/04/2006 10:04:00,Psein 830 is rood gedoofd.,Psein 830 is rood gedoofd.,950.0,10688510.0,SEIN,...,08/04/2006,16:05:00,359.0,08/04/2006,16:05:00,doorbranden,PS,NaN,RSCWA,NaN
89798,908084,99002830,103.0,KFH,08/04/2006 20:34:00,Whv-Brdv spoor MB-MC thv km 201.2 vrij toegan...,Whv-Brdv spoor MB-MC thv km 201.2 vrij t,950.0,10107663.0,BAANLICHAA,...,08/04/2006,21:35:00,54.0,08/04/2006,21:47:00,NaN,NaN,NaN,BRDV,NaN


In [5]:
#uitgeschreven lijsten zodat ik ze niet steeds hoef over te typen

# melding
df[['stm_sap_meldtekst', 'stm_geo_mld', 'stm_oorz_code', 'stm_prioriteit']]
# tijd
df[['stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_aanngeb_dd', 'stm_aanngeb_tijd', 'stm_aanntpl_dd', 'stm_aanntpl_tijd']]
df[['stm_fh_dd', 'stm_fh_tijd', 'stm_fh_duur', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd']]
# equipment
df[['stm_equipm_omschr_mld', 'stm_equipm_nr_mld']]
# locatie
df[['stm_km_van_mld', 'stm_km_tot_mld', 'stm_vl_post', 'stm_contractgeb_mld', 'stm_pplg_van', 'stm_pplg_naar', 'stm_dstrglp_van', 'stm_dstrglp_naar']]
# prognose
df[['stm_progfh_in_datum', 'stm_progfh_in_tijd', 'stm_progfh_in_invoer_dat', 'stm_progfh_in_invoer_tijd', 'stm_progfh_in_duur']]


,stm_progfh_in_datum,stm_progfh_in_tijd,stm_progfh_in_invoer_dat,stm_progfh_in_invoer_tijd,stm_progfh_in_duur
0,NaN,00:00:00,NaN,00:00:00,99999999.0
1,NaN,00:00:00,NaN,00:00:00,99999999.0
2,NaN,00:00:00,NaN,00:00:00,99999999.0
3,NaN,00:00:00,NaN,00:00:00,99999999.0
4,NaN,00:00:00,NaN,00:00:00,99999999.0
...,...,...,...,...,...
89850,10/05/2013,14:00:00,10/05/2013,13:34:37,113
89851,10/05/2013,15:45:00,15/05/2013,09:43:15,15
89852,10/06/2013,16:00:00,13/05/2013,11:29:55,40598
89853,10/05/2013,19:30:00,10/05/2013,19:34:59,4


# Baseline

In [6]:
begin_eind_tijd_dirty = df[['stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_sap_storeinddatum', 'stm_sap_storeindtijd']].dropna()
begin_eind_tijd # begin en eindtijd in dataframe gezet.

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd
1,03/01/2006,19:18:00,03/01/2006,19:42:00
2,03/01/2006,13:33:00,03/01/2006,13:34:00
3,03/01/2006,10:32:00,03/01/2006,11:25:00
4,04/01/2006,12:27:00,04/01/2006,12:27:00
5,10/01/2006,22:25:00,10/01/2006,23:10:00
...,...,...,...,...
89850,10/05/2013,11:30:00,10/05/2013,13:34:00
89851,10/05/2013,14:00:00,10/05/2013,15:41:00
89852,10/05/2013,18:39:00,03/07/2013,05:42:00
89853,10/05/2013,19:22:00,10/05/2013,19:30:00


In [23]:
begin_eind_tijd_clean = begin_eind_tijd.loc[(begin_eind_tijd['stm_sap_meldtijd'] != '::') & (begin_eind_tijd['stm_sap_storeindtijd'] != '::')]

In [24]:
stm_sap_melddatum_tijd_object = begin_eind_tijd_clean['stm_sap_melddatum'] + ' ' + begin_eind_tijd_clean['stm_sap_meldtijd']
stm_sap_melddatum_tijd = pd.DataFrame(stm_sap_melddatum_tijd_object,
                   columns=['Begin_tijd_en_datum']) # datum en tijd samengevoegd voor het makkelijker rekenen

In [25]:
stm_sap_einddatum_tijd_object = begin_eind_tijd_clean['stm_sap_storeinddatum'] + ' ' + begin_eind_tijd_clean['stm_sap_storeindtijd']
stm_sap_einddatum_tijd = pd.DataFrame(stm_sap_einddatum_tijd_object,
                   columns=['Eind_tijd_en_datum']) # datum en tijd samengevoegd voor het makkelijker rekenen

In [54]:
result_begin_eind_tijd = pd.concat([begin_eind_tijd, stm_sap_melddatum_tijd, stm_sap_einddatum_tijd],axis=1).dropna()
result_begin_eind_tijd # de samengevoegde waarde in de dataframe gezet

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum
1,03/01/2006,19:18:00,03/01/2006,19:42:00,03/01/2006 19:18:00,03/01/2006 19:42:00
2,03/01/2006,13:33:00,03/01/2006,13:34:00,03/01/2006 13:33:00,03/01/2006 13:34:00
3,03/01/2006,10:32:00,03/01/2006,11:25:00,03/01/2006 10:32:00,03/01/2006 11:25:00
4,04/01/2006,12:27:00,04/01/2006,12:27:00,04/01/2006 12:27:00,04/01/2006 12:27:00
5,10/01/2006,22:25:00,10/01/2006,23:10:00,10/01/2006 22:25:00,10/01/2006 23:10:00
...,...,...,...,...,...,...
89850,10/05/2013,11:30:00,10/05/2013,13:34:00,10/05/2013 11:30:00,10/05/2013 13:34:00
89851,10/05/2013,14:00:00,10/05/2013,15:41:00,10/05/2013 14:00:00,10/05/2013 15:41:00
89852,10/05/2013,18:39:00,03/07/2013,05:42:00,10/05/2013 18:39:00,03/07/2013 05:42:00
89853,10/05/2013,19:22:00,10/05/2013,19:30:00,10/05/2013 19:22:00,10/05/2013 19:30:00


In [55]:
result_begin_eind_tijd['Begin_tijd_en_datum'] = pd.to_datetime(result_begin_eind_tijd['Begin_tijd_en_datum'])
result_begin_eind_tijd['Eind_tijd_en_datum'] = pd.to_datetime(result_begin_eind_tijd['Eind_tijd_en_datum'])

In [56]:
fix_duur = result_begin_eind_tijd['Eind_tijd_en_datum'] - result_begin_eind_tijd['Begin_tijd_en_datum']

In [57]:
Duur_fix = pd.DataFrame(fix_duur,
                   columns=['Duur fix'])

In [60]:
Final_begin_eind_tijd = pd.concat([result_begin_eind_tijd, Duur_fix],axis=1).dropna()
Final_begin_eind_tijd

,stm_sap_melddatum,stm_sap_meldtijd,stm_sap_storeinddatum,stm_sap_storeindtijd,Begin_tijd_en_datum,Eind_tijd_en_datum,Duur fix
1,03/01/2006,19:18:00,03/01/2006,19:42:00,2006-03-01 19:18:00,2006-03-01 19:42:00,0 days 00:24:00
2,03/01/2006,13:33:00,03/01/2006,13:34:00,2006-03-01 13:33:00,2006-03-01 13:34:00,0 days 00:01:00
3,03/01/2006,10:32:00,03/01/2006,11:25:00,2006-03-01 10:32:00,2006-03-01 11:25:00,0 days 00:53:00
4,04/01/2006,12:27:00,04/01/2006,12:27:00,2006-04-01 12:27:00,2006-04-01 12:27:00,0 days 00:00:00
5,10/01/2006,22:25:00,10/01/2006,23:10:00,2006-10-01 22:25:00,2006-10-01 23:10:00,0 days 00:45:00
...,...,...,...,...,...,...,...
89850,10/05/2013,11:30:00,10/05/2013,13:34:00,2013-10-05 11:30:00,2013-10-05 13:34:00,0 days 02:04:00
89851,10/05/2013,14:00:00,10/05/2013,15:41:00,2013-10-05 14:00:00,2013-10-05 15:41:00,0 days 01:41:00
89852,10/05/2013,18:39:00,03/07/2013,05:42:00,2013-10-05 18:39:00,2013-03-07 05:42:00,-213 days +11:03:00
89853,10/05/2013,19:22:00,10/05/2013,19:30:00,2013-10-05 19:22:00,2013-10-05 19:30:00,0 days 00:08:00


In [63]:
Final_begin_eind_tijd['Duur fix'].describe()

count                          66242
mean       4 days 04:27:07.133314211
std      276 days 09:36:33.368599360
min              -914 days +20:42:59
25%                  0 days 01:11:00
50%                  0 days 02:55:00
75%           0 days 22:55:30.750000
max              69425 days 17:43:26
Name: Duur fix, dtype: object